# Homework 3

## Consider the bivariate normal distribution we introduced in the class. Let's make the assumptions that  $\mu_x = 0$, $\mu_y = \mu_y$, $\sigma_x = \sigma_x$, $\sigma_y = 1$, and $\rho=\rho$.

### Write down the conditional distributions of X and Y. Note that both of them would be normal distributions. (See Section 1.4.3 and 1.4.3.1 of the lecture note.)

\begin{align}
(X|Y=y) & \sim N\left( \rho \sigma_x(y-\mu_y),\quad \sigma_x^2(1-\rho^2) \right),\\
(Y|X=x) & \sim N\left( \mu_y + \rho \frac{x}{\sigma_x},\quad (1-\rho^2) \right).
\end{align}

### Write a Julia function of Gibbs sampler that allows users to choose: the number of sampling points $N$, the values of $\mu_y$, $\sigma_x$, $\rho$, and initial values of $(x_0, y_0)$, and the number of burn-in points. You should follow the steps in Section 1.3 of the lecture note to draw $(x_n, y_n), n=1,\ldots,N$.


In [2]:
using Random, Statistics, Distributions

function Gibbs_BN(; N=100, μy::Real, σx::Real, rho=0.1, init=[2,2], skip=0) # total length: 1+skip+N; "1" for initial value
    # skip is for burn-in
    x = ones(1+skip+N) 
    y = ones(1+skip+N)
    x[1]=init[1] 
    y[1]=init[2]
    
    for n = 2:1+skip+N  # 保留第一個位子給init
        x[n] = rand(   Normal(rho*(σx)*(y[n-1]-μy), sqrt((σx)^2 * (1-rho^2)))) 
        
        y[n] = rand(   Normal( (μy)+rho*x[n]/(σx), sqrt(1-rho^2) ) ) 
    end
    return x[1+skip+1:end], y[1+skip+1:end] 
end

x, y = Gibbs_BN(N=100000, μy=0 ,σx=1 ,rho=0.5, init=[2,2], skip=100)


[mean(x) std(x)] |> display
[mean(y) std(y)] |> display
cor(x,y) 

1×2 Matrix{Float64}:
 -0.00156272  1.00257

1×2 Matrix{Float64}:
 -0.00412299  1.00418

0.5019276563232484

### Draw a graph  similar to the one we showed in the class. Remember to add sliders for both $N$ and $\rho$.

In [3]:
using Interact, WebIO
using Distributions, Plots, LinearAlgebra, Statistics

min_N = 50
max_N = 2000

@manipulate for num=min_N:max_N, corr=-1:0.1:1

    x, y = Gibbs_BN(N=num, μy=0, σx=1, rho=corr, init=[2,2], skip=100)

    layout = @layout [a{0.7w,0.4h} _
                      b{0.6w,0.6h} c{0.4w, 0.6h}]

    default(fillcolor=:purple, markercolor=:pink, grid=false, legend=false)
    plot(layout=layout, link=:both, size=(400, 400),  margin=-10Plots.pt)
    scatter!(x,y, subplot=2, framestyle =:box)
    histogram!([x y], subplot=[1 3], orientation=[:v :h], framestyle=:none, bins=min(num, 100), normalize=true)
end

Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Scope(Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :label), Any["num"], Dict{Symbol, Any}(:className => "interact ", :style => Dict{Any, Any}(:padding => "5px 10px 0px 10px")))], Dict{Symbol, Any}(:className => "interact-flex-row-left")), Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :input), Any[], Dict{Symbol, Any}(:max => 2000, :min => 50, :attributes => Dict{Any, Any}(:type => "range", Symbol("data-bind") => "numericValue: index, valueUpdate: 'input', event: {change: function (){this.changes(this.changes()+1)}}", "orient" => "horizontal"), :step => 1, :className => "slider slider is-fullwidth", :style => Dict{Any, Any}()))], Dict{Symbol, Any}(:className => "interact-flex-row-center")), Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :p), Any[], Dict{Symbol, Any}(:attributes => Dict("data-bind" => "text: formatted_val")))], Dict{Symbol, Any}(:className => "interact-flex-row-right"))], Dict{Symbol, Any}(:className => "interact-flex-row interact-widget")), Dict{String, Tuple{AbstractObservable, Union{Nothing, Bool}}}("changes" => (Observable(0), nothing), "index" => (Observable(1025), nothing)), Set{String}(), nothing, Asset[Asset("js", "knockout", "/Users/weishangjie/.julia/packages/Knockout/HReiN/src/../assets/knockout.js"), Asset("js", "knockout_punches", "/Users/weishangjie/.julia/packages/Knockout/HReiN/src/../assets/knockout_punches.js"), Asset("js", nothing, "/Users/weishangjie/.julia/packages/InteractBase/Qhvxg/src/../assets/all.js"), Asset("css", nothing, "/Users/weishangjie/.julia/packages/InteractBase/Qhvxg/src/../assets/style.css"), Asset("css", nothing, "/Users/weishangjie/.julia/packages/Interact/PENUy/src/../assets/bulma_confined.min.css")], Dict{Any, Any}("changes" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"changes\"]()) ? (this.valueFromJulia[\"changes\"]=true, this.model[\"changes\"](val)) : undefined})")], "index" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"index\"]()) ? (this.valueFromJulia[\"index\"]=true, this.model[\"index\"](val)) : undefined})")]), WebIO.ConnectionPool(Channel{Any}(32), Set{AbstractConnection}(), Condition(Base.InvasiveLinkedList{Task}(Task (runnable) @0x0000000162bc5b60, Task (runnable) @0x0000000162bc5b60), Base.AlwaysLockedST(1))), WebIO.JSString[WebIO.JSString("function () {\n    var handler = (function (ko, koPunches) {\n    ko.punches.enableAll();\n    ko.bindingHandlers.numericValue = {\n        init: function(element, valueAccessor, allBindings, data, context) {\n            var stringified = ko.observable(ko.unwrap(valueAccessor()));\n            stringified.subscribe(function(value) {\n                var val = parseFloat(value);\n                if (!isNaN(val)) {\n                    valueAccessor()(val);\n                }\n            });\n            valueAccessor().subscribe(function(value) {\n                var str = JSON.stringify(value);\n                if ((str == \"0\") && ([\"-0\", \"-0.\"].indexOf(stringified()) >= 0))\n                     return;\n                 if ([\"null\", \"\"].indexOf(str) >= 0)\n                     return;\n                stringified(str);\n            });\n            ko.applyBindingsToNode(\n                element,\n                {\n                    value: stringified,\n                    valueUpdate: allBindings.get('valueUpdate'),\n                },\n                context,\n            );\n        }\n    };\n    var json_data = {\"formatted_vals\":[\"50\",\"51\",\"52\",\"53\",\"54\",\"55\",\"56\",\"57\",\"58\",\"59\",\"60\",\"61\",\"62\",\"63\",\"64\",\"65\",\"66\",\"67\",\"68\",\"69\",\"70\",\"71\",\"72\",\"73\",\"74\",\"75\",\"76\",\"77\",\"78\",\"79\",\"80\",\"81\",\"82\",\"83\",\"84\",\"85\",\"86\",\"87\",\"88\",\"89\",\"90\",\"91\",\"92\",\"93\",\"94